In [ ]:
import torch
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, GATConv, GATv2Conv, TopKPooling
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid, TUDataset
from torch_geometric.loader import DataLoader
from torch_scatter import scatter_mean

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

In [ ]:
data = Data(x=x, edge_index=edge_index)

In [ ]:
data

In [ ]:
data.keys

In [ ]:
data["x"]

In [ ]:
data.num_nodes

In [ ]:
data.num_edges

In [ ]:
data.num_node_features

In [ ]:
data.has_isolated_nodes()

In [ ]:
data.has_self_loops()

In [ ]:
data.is_directed()

In [ ]:
dataset = TUDataset(root="datasets/ENZYMES", name="ENZYMES", use_node_attr=True)

In [ ]:
dataset

In [ ]:
dataset.name

In [ ]:
len(dataset)

In [ ]:
dataset.num_classes

In [ ]:
dataset.num_node_features

In [ ]:
dataset[0].is_undirected()

In [ ]:
dataset = dataset.shuffle()

In [ ]:
train_dataset = dataset[:540]
test_dataset = dataset[540:]

In [ ]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
for batch in loader:
    print(batch.num_graphs)

In [ ]:
for data in loader:
    x = scatter_mean(data.x, data.batch, dim=0)
    print(x.size())

In [ ]:
dataset = Planetoid(root="datasets/Cora", name="Cora")

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training, p=0.1)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(400):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f"Accuracy: {acc}")

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(dataset.num_node_features, 16, heads=8, dropout=0.2, concat=True)
        self.conv2 = GATConv(16*8, dataset.num_classes, heads=1, concat=False, dropout=0.2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training, p=0.2)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GAT().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f"Accuracy: {acc}")

In [ ]:
class GATv2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATv2Conv(dataset.num_node_features, 16, heads=8, dropout=0.2, concat=True)
        self.conv2 = GATv2Conv(16*8, dataset.num_classes, heads=1, dropout=0.2, concat=False)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training, p=0.2)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GATv2().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f"Accuracy: {acc}")

In [ ]:
class GraphUNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.depth = 4
        self.enc_convs = torch.nn.ModuleList()
        self.dec_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()

        self.enc_convs.append(GCNConv(dataset.num_node_features, 16, improved=True))
        for _ in range(self.depth):
            self.pools.append(TopKPooling(16, 0.5))
            self.enc_convs.append(GCNConv(16, 16, improved=True))

        for _ in range(self.depth - 1):
            self.dec_convs.append(GCNConv(16, 16, improved=True))
        self.dec_convs.append(GCNConv(16, dataset.num_classes, improved=True))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.enc_convs[0](x, edge_index))
        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, _, perm, _ = self.pools[i - 1](x, edge_index)
            x = F.relu(self.enc_convs[i](x, edge_index))
            x = F.dropout(x, training=self.training, p=0.1)

            perms += [perm]
            if i < self.depth:
                xs += [x]
                edge_indices += [edge_index]

        for i in range(self.depth):
            j = self.depth - i - 1
            res, edge_index, perm = xs[j], edge_indices[j], perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up

            x = self.dec_convs[i](x, edge_index)
            x = F.relu(x) if i < self.depth - 1 else x
            x = F.dropout(x, training=self.training, p=0.1) if i < self.depth - 1 else x

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GraphUNet().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f"Accuracy: {acc}")